## I'll be working with laptops.csv, a CSV file containing information on about 1,300 laptop computers:
#### Objectives:
 - Clean columns names
 - Extract and convert numeric values from string values
 - Extract string data
 - Work with missing values

In [30]:
import numpy as np
import pandas as pd
laptops = pd.read_csv('laptops.csv', encoding='Windows-1251')
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


#### Creating a function that uses Python string methods to clean column labels, and then use list comprehension to apply that function to each label

In [31]:
def clean_col(col):
    col = col.strip()
    col = col.replace("Operating System","os")
    col = col.replace(" ","_")
    col = col.replace("(","")
    col = col.replace(")","")
    col = col.lower()
    return col
laptops.columns = [clean_col(c) for c in laptops.columns]

#### Converting the screen_size, ram, weight and price_euros column to numeric

In [32]:
laptops["screen_size"] = laptops["screen_size"].str.replace('"','').astype(float)
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)
laptops['ram'] = laptops['ram'].str.replace('GB','').astype(int)
laptops.rename({'ram':'ram_gb'}, axis = 1, inplace= True)
dtypes = laptops.dtypes
print(laptops["screen_size_inches"].value_counts())

15.6    665
14.0    197
13.3    164
17.3    164
12.5     39
11.6     33
13.9      6
13.5      6
12.0      6
12.3      5
15.4      4
10.1      4
15.0      4
13.0      2
18.4      1
14.1      1
17.0      1
11.3      1
Name: screen_size_inches, dtype: int64


In [33]:
laptops["weight"] =(laptops["weight"].str.replace("kg",""))
laptops["weight"] =(laptops["weight"].str.replace("s",""))
laptops["weight"] =(laptops["weight"].astype(float))
laptops.rename({'weight':'weight_kg'}, axis = 1, inplace= True)
laptops['price_euros'] =(laptops['price_euros'].str.replace(',','.').astype(float))
weight_describe = laptops['weight_kg'].describe()
price_describe = laptops['price_euros'].describe()
print(laptops["weight_kg"].value_counts())

2.200    126
2.100     58
2.000     45
2.400     44
2.300     41
2.500     38
2.800     28
1.200     26
1.860     25
1.900     24
2.040     24
1.400     24
2.180     23
1.500     21
1.600     20
1.700     19
1.950     18
1.800     17
1.300     17
1.100     17
3.000     16
1.260     15
1.360     14
1.910     14
1.370     13
2.620     12
2.600     12
1.250     12
2.700     12
1.630     11
        ... 
1.550      1
3.520      1
0.990      1
2.750      1
1.590      1
4.500      1
3.250      1
1.940      1
2.830      1
1.140      1
1.410      1
4.700      1
2.380      1
2.191      1
3.800      1
2.591      1
1.190      1
4.400      1
0.910      1
3.420      1
2.790      1
2.990      1
2.260      1
2.210      1
1.180      1
1.310      1
2.720      1
3.600      1
3.740      1
4.330      1
Name: weight_kg, Length: 171, dtype: int64


#### Extracting the manufacturer name from the gpu column

In [34]:
laptops["gpu_manufacturer"] = (laptops["gpu"]
                                    .str.split(n=1,expand=True)
                                    .iloc[:,0]
                               )
laptops['cpu_manufacturer'] = (laptops['cpu']
                               .str.split(n=1,expand=True)
                               .iloc[:,0]
                              )
print(laptops["cpu_manufacturer"].value_counts())

Intel      1240
AMD          62
Samsung       1
Name: cpu_manufacturer, dtype: int64


#### Extracting cpu_speed_ghz and screen_resolution

In [49]:
screen_res = laptops["screen"].str.rsplit(n=1, expand=True)
screen_res.columns = ["A", "B"]
screen_res.loc[screen_res["B"].isnull(), "B"] = screen_res["A"]
laptops["screen_resolution"] = screen_res["B"]
laptops['cpu_speed_ghz']=(laptops['cpu']
                          .str.replace('GHz','')
                          .str.rsplit(n=1, expand = True)
                          .iloc[:,1].astype(float)
                         )
print(laptops["cpu_speed_ghz"].value_counts())
print(laptops["screen_resolution"].value_counts())

2.50    293
2.70    166
2.80    165
1.60    134
2.00     86
2.30     86
1.80     78
2.60     76
1.10     53
2.40     52
2.90     21
3.00     19
1.20     15
1.44     12
2.20     11
1.50     10
1.30      6
3.60      5
0.90      4
2.10      3
3.10      3
1.90      2
1.92      1
1.00      1
3.20      1
Name: cpu_speed_ghz, dtype: int64
1920x1080    841
1366x768     308
3840x2160     43
3200x1800     27
2560x1440     23
1600x900      23
2560x1600      6
2304x1440      6
2256x1504      6
1920x1200      5
2880x1800      4
2400x1600      4
1440x900       4
2160x1440      2
2736x1824      1
Name: screen_resolution, dtype: int64


#### Correcting values in the os column

In [35]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}
laptops['os'] = laptops['os'].map(mapping_dict)

#### Removing rows and columns from the laptops dataframe that have null values

In [36]:
laptops_no_null_rows = laptops.dropna(axis = 0)
laptops_no_null_cols = laptops.dropna(axis = 1)

#### Assigning the value Version Unknown to the os_version column with value No OS

In [37]:
value_counts_before = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"
laptops.loc[laptops['os'] == 'No OS','os_version'] = 'Version Unknown'
value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()

#### Cleaning the Storage Column

In [38]:
laptops['storage'] =(laptops['storage'].str.replace('+',''))
laptops['storage'] = (laptops['storage'].str.replace('  ',''))
laptops['storage'] = (laptops['storage'].str.replace('GB',''))
laptops['storage'] = (laptops['storage'].str.replace('TB','000'))
laptops['storage'] = (laptops['storage'].str.replace('Flash Storage','Flash_Storage'))
storage_1_capacity_gb = (laptops['storage']
                          .str.split(n=1, expand = True)
                          .iloc[:,0]
                         )
storage_1_type = (laptops['storage']
                          .str.split(n=2, expand = True)
                          .iloc[:,1]
                         )
storage_1_type = (storage_1_type.str.replace('Flash_Storage','Flash Storage'))
storage_2_capacity_gb = (laptops['storage']
                          .str.split(n=3, expand = True)
                          .iloc[:,2]
                         )
storage_2_type = (laptops['storage']
                          .str.split(n=4, expand = True)
                          .iloc[:,3]
                         )
laptops['storage_1_capacity_gb'] = storage_1_capacity_gb.astype(float)
laptops['storage_1_type'] = storage_1_type
laptops['storage_2_capacity_gb'] = storage_2_capacity_gb.astype(float)
laptops['storage_2_type'] = storage_2_type
laptops.drop(["storage"], axis=1, inplace=True)

#### Re-ordering the columns and saving the cleaned CSV

In [50]:
laptops_dtypes = laptops.dtypes
cols = ['manufacturer', 'model_name', 'category', 'screen_size_inches',
        'screen', 'cpu', 'cpu_manufacturer', 'screen_resolution', 'cpu_speed_ghz', 'ram_gb',
        'storage_1_type', 'storage_1_capacity_gb', 'storage_2_type',
        'storage_2_capacity_gb', 'gpu', 'gpu_manufacturer', 'os',
        'os_version', 'weight_kg', 'price_euros']
laptops = laptops[cols]
laptops.to_csv('laptops_cleaned.csv',index=False)
laptops_cleaned = pd.read_csv('laptops_cleaned.csv')
laptops_cleaned_dtypes = laptops_cleaned.dtypes